### Compute Homologous Pair Statistics
This notebook uses the defined neuron and lineage homologies from annotations and the `BrainPairCensus` annotation to measure three kinds of properties for matched neurons.
    1) Lineage assignment and soma-soma distance
    2) NBLAST score
    3) Connectivity to identified homologous neurons

In [ ]:
import sys
# SET THE PATHS APPROPRIATELY
sys.path.append('path_to_catpy')
sys.path.append('path_to_catalysis')
epsilon = sys.float_info.epsilon

import catalysis as cat
import catalysis.pynblast as pynblast
import catalysis.plt as catplt
import catalysis.transform as transform
import catalysis.completeness as completeness

import plotly.offline as py
import plotly.graph_objs as go

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as cl

import numpy as np
import scipy as sp
import pandas as pd
import networkx as nx
import re

import tqdm
import pickle
from itertools import chain

from importlib import reload

In [ ]:
# Set this with the correct information following the template in data/example_project_info.json.
my_json_project_file =  'my_json_project_file.json'
l1data = cat.CatmaidDataInterface.from_json( my_json_project_file )

The following pulls all lineage annotations, in this case based on particular string matches. This could be replaced down the road with, e.g., meta-annotations.

In [ ]:
wb_match = re.compile('\*.*akira')
lineage_parser = re.compile('\*(?P<group>.*?)_(?P<instance>[rl]) akira')
hemilateral_groups = l1data.match_groups_from_select_annotations( wb_match, lineage_parser )
lineages = sorted(list(hemilateral_groups.keys()))

# Let's shove the sensory neurons in here too.
hemilateral_groups['sensory'] = {'l':'Brain&SEZ sensory left akira','r':'Brain&SEZ sensory right akira'}
lineages.append('sensory')

left_names = [hemilateral_groups[lin]['l'] for lin in hemilateral_groups if len(hemilateral_groups[lin])>1]
right_names = [hemilateral_groups[lin]['r'] for lin in hemilateral_groups if len(hemilateral_groups[lin])>1 ]

left_anno_ids = l1data.get_annotation_ids_from_names(left_names)
right_anno_ids = l1data.get_annotation_ids_from_names(right_names)

In [ ]:
left_skids = l1data.get_ids_from_annotations(left_anno_ids,flatten=True)
right_skids = l1data.get_ids_from_annotations(right_anno_ids,flatten=True)

id_pairs_matched = []
id_pairs_ipsi = []
id_pairs_contra = []

for lin in sorted(hemilateral_groups):
    print(lin)
    if len(hemilateral_groups[lin]) > 1:
        if lin == 'sensory':
            ipm, ipi, ipc = completeness.make_id_pairs(l1data,
                hemilateral_groups['sensory']['l'],
                hemilateral_groups['sensory']['r'],
                'BrainPairCensus',
                max_open_ends=0.05,
                min_node_count=300,
                sensory_exception=True)
        else:
            ipm, ipi, ipc = completeness.make_id_pairs(l1data,
                            hemilateral_groups[lin]['l'],
                            hemilateral_groups[lin]['r'],
                            'BrainPairCensus')
        id_pairs_matched.append(ipm)
        id_pairs_ipsi.append(ipi)
        id_pairs_contra.append(ipc)

In [ ]:
# A list of id pairs on left and right that are from homologous neurons.
id_pairs_matched_l = [pair for pair in chain.from_iterable(id_pairs_matched)]

# A list of ids pairs within the same side that are in the same lineage, but not homologous neurons.
id_pairs_ipsi_l = [pair for pair in chain.from_iterable(id_pairs_ipsi)]

# A list of id pairs on left and right that are the homologous lineage, but not the homologous neuron.
id_pairs_contra_l = [pair for pair in chain.from_iterable(id_pairs_contra)]

### Save all matched pairs if you're happy wiht the analysis, since as you've seen it takes a while to compute.
##### (Optional)


In [ ]:
with open('id_pairs_all', 'wb') as fid:
    pickle.dump({'matched': id_pairs_matched_l, 'ipsi':id_pairs_ipsi_l, 'contra': id_pairs_contra_l}, fid)

---
## Measure properties of matched neurons

#### 1. Compare the anatomical similarity of matched neurons and unmatched neurons after landmark-based transformation.

In [ ]:
# Import an NBLAST distance mat your favorite way
adult_fly_f = pd.read_csv("catalysis/data/smat_jefferis.csv",delimiter=' ')
smat = pynblast.ScoreMatrixLookup.from_dataframe( adult_fly_f )
# Do this to reduce things to L1 volume size and change scale to nm, based on scale up observed in the L1/L3 data papers and desired properties
smat.d_range = smat.d_range * 1000 / 1.5

In [ ]:
pair_ids = [skid for skid in chain.from_iterable(id_pairs_matched_l)]
pair_nrns = cat.NeuronList.from_id_list(id_list=pair_ids,CatmaidInterface=l1data)

##### For left/right homologous pair, compute the NBLAST similarity and the distance between somata after transformation

In [ ]:
S_match = dict()
Dsoma_match = dict()
for pair in tqdm.tqdm(id_pairs_matched_l):
    nrn_q = pair_nrns[pair[0]]
    nrn_t = transform.transform_neuron_from_landmarks(pair_nrns[pair[1]],
                                    from_group='brain hemisphere right',
                                    to_group='brain hemisphere left',
                                    CatmaidInterface=l1data,
                                    contralateral=True )
    S_match[str(pair)] = pynblast.exact_nblast(smat, [nrn_q], [nrn_t] ).values[0][0]

    if not completeness.fragment_test(nrn_q) and not completeness.fragment_test(nrn_t):
        xyz_q = np.array(nrn_q.nodeloc[ nrn_q.tags['soma'][0] ])
        xyz_t = np.array(nrn_t.nodeloc[ nrn_t.tags['soma'][0] ])
        Dsoma_match[str(pair)] = np.linalg.norm(xyz_q-xyz_t)

##### For every left/right same-lineage, but non-homologous pair, compute the NBLAST similarity and the distance between somata after transformation

In [ ]:
ids_contra = list(set([skid for skid in chain.from_iterable(id_pairs_contra_l)]))
print(len(ids_contra))
nrns_contra = cat.NeuronList.from_id_list(id_list=ids_contra,CatmaidInterface=l1data)

In [ ]:
S_contra = dict()
Dsoma_contra = dict()

vals = np.random.choice(len(id_pairs_contra_l), 300 )

for val in tqdm.tqdm(vals):
    pair = id_pairs_contra_l[val]
    nrn_q = nrns_contra[pair[0]]
    nrn_t = transform.transform_neuron_from_landmarks( nrns_contra[pair[1]],
                                    from_group='brain hemisphere right',
                                    to_group='brain hemisphere left',
                                    CatmaidInterface=l1data,
                                    contralateral=True )

    S_contra[str(pair)] = pynblast.exact_nblast(smat, [nrn_q], [nrn_t] ).values[0][0]

    if not completeness.fragment_test(nrn_q) and not completeness.fragment_test(nrn_t):
        xyz_q = np.array(nrn_q.nodeloc[ nrn_q.tags['soma'][0] ])
        xyz_t = np.array(nrn_t.nodeloc[ nrn_t.tags['soma'][0] ])
        Dsoma_contra[str(pair)] = np.linalg.norm(xyz_q-xyz_t)

### Plot the soma distance and similarity scores as a check

In [ ]:
sns.kdeplot(np.array([d for d in Dsoma_match.values()])/1000,cumulative=True,cut=0)
sns.kdeplot(np.array([d for d in Dsoma_contra.values()])/1000,cumulative=True,cut=0)

In [ ]:
sns.kdeplot(np.array([d for d in S_match.values()]),cumulative=True,cut=0)
sns.kdeplot(np.array([d for d in S_contra.values()]),cumulative=True,cut=0)

---
### Get p(matched | S, d_soma ), p( unmatched | S, d_soma ), p( S ), and p( d_soma )
In order to get the basic statistics, we need to generate three different lists of id pairs of *complete* neurons only.

1. Matched partners.
2. Same lineage, same side.
3. Same lineage, opposite side but unmatched.

Because the data is somewhat limited (and basically doesn't scale to values of similarity that are indeed possible to achieve), we're going to take a model based approached.

##### First, fit likelihood based on NBLAST similarity with logistic function.
Note: The drop-off in match probability for very high values of similarity is clearly non-realistic, but just means that we don't usually see such high values in our data. Nonetheless, let's use a logistic function approximation to be robust to the chance we do see higher values.

In [ ]:
Sdens_match = sp.stats.gaussian_kde([d for d in S_match.values()])
Sdens_contra = sp.stats.gaussian_kde([d for d in S_contra.values()])
#Sdens_ipsi = sp.stats.gaussian_kde([d for d in S_ipsi.values()])

# Note that the choice of fit functions is explicitly connected to pynblast._logistic_ratio()
fitfun = lambda p, x : p[0] / ( 1 + np.exp( -1 * p[1] * (x - p[2]) ) )
errfun = lambda p, x, y: fitfun(p,x) - y

xs = np.arange(0,0.8, 0.01)
pair_vals = Sdens_match.evaluate(xs)
tot_vals_contra = Sdens_match.evaluate(xs)+Sdens_contra.evaluate(xs)
dat = pair_vals/tot_vals_contra

p0 = [0.9, 1, 0.6]
shape_fit_param, success = sp.optimize.leastsq( errfun, p0, args=(xs, dat) )

## Check the fit quality

xs = np.arange(0,1,0.02)
pair_vals = Sdens_match.evaluate(xs)
tot_vals_contra = Sdens_match.evaluate(xs)+Sdens_contra.evaluate(xs)
non_match_contra = Sdens_contra.evaluate(xs)

plt.plot(xs,pair_vals / tot_vals_contra)
plt.plot(xs, fitfun(shape_fit_param,xs))

#### Fit the soma distances

Here, our model is a bit more complicated. The matched and non-matched soma distance distribution can be well-fit with gamma distributions.
However, if that's all we do, then extremely close soma distances are treated as a bit stronger evidence than we observe in the data (it's highly suggestive, but will never prove anything).
We thus add a short-range expontential weakening of the likelihood function to give a better heuristic fit to the data.

In [ ]:
Ddens_match = sp.stats.gaussian_kde([d for d in Dsoma_match.values()])
Ddens_contra = sp.stats.gaussian_kde([d for d in Dsoma_contra.values()])

xs = np.arange(0,46000,1000)
pair_vals = Ddens_match.evaluate(xs)
Dtot_vals_contra = Ddens_match.evaluate(xs)+Ddens_contra.evaluate(xs)

match_param = sp.stats.gamma.fit(list(Dsoma_match.values()), floc=0 )
contra_param = sp.stats.gamma.fit(list(Dsoma_contra.values()), floc=0 )

xval = np.arange(0,30000,1000)
yval = sp.stats.gamma.pdf(xval, *match_param )
sns.distplot(list(Dsoma_match.values()))
plt.plot(xval,yval)

xvalc = np.arange(0,30000,1000)
yvalc = sp.stats.gamma.pdf(xvalc, *contra_param )
sns.distplot(list(Dsoma_contra.values()))
plt.plot(xvalc,yvalc)
plt.xlabel('Soma Distance')

In [ ]:
# Again, note that the following is explicitly tied to pynblast._gamma_ratio()
def gamma_odds( xs, num_param, denom_param, exp_param ):
    num = sp.stats.gamma.pdf( xs, *num_param )
    denom = sp.stats.gamma.pdf( xs, *denom_param)
    return num / (num+denom) * ( 1-exp_param[0] * np.exp(-xs/exp_param[1]))

In [ ]:
fitfun_gr = lambda p, x : gamma_odds( x[0], x[1], x[2],  p)
errfun_gr = lambda p, x, y: fitfun_gr(p,x) - y

xs = np.arange(1,40000,100)
pair_vals = Ddens_match.evaluate(xs)
Dtot_vals_contra = Ddens_match.evaluate(xs)+Ddens_contra.evaluate(xs)
dat = pair_vals / Dtot_vals_contra

p_gr0 = (0.2, 2000)
exp_param, success = sp.optimize.leastsq( errfun_gr, p_gr0, args=( (xs,match_param,contra_param) , dat ) )

plt.plot(xs, pair_vals / Dtot_vals_contra)
plt.plot(xs, gamma_odds( xs, match_param, contra_param, exp_param ) )

#### Save the fits that we just did for future comparisons

In [ ]:
with open('morpho_stats','wb') as fid:
    pickle.dump({'shape':shape_fit_param,
                 'dist_match':match_param,
                 'dist_nonmatch':contra_param,
                 'exp_cutoff':exp_param}, fid)

---
## Look at connection weights between paired neurons

For every complete neuron in the pair list, find its complete synaptic partners.
For every complete partner that is on the pair list, find the matched connection as well.
We will do this separately for upstream and downstream.

But also, compute P(n_syn) for all connections, so we can compare P(n_syn_1,n_syn_2) to P(n_syn_1)P(n_syn_2)

Start with the homologous neuron id pairs based on the meta-annotation `BrainPairCensus` that have been completed to a satisfactory degree.

In [ ]:
left_annos = [hemilateral_groups[lin]['l'] for lin in hemilateral_groups if len(hemilateral_groups[lin])>1]
right_annos = [hemilateral_groups[lin]['r'] for lin in hemilateral_groups if len(hemilateral_groups[lin])>1]

all_left_ids = l1data.get_ids_from_annotations(left_annos, flatten=True)
all_right_ids = l1data.get_ids_from_annotations(right_annos, flatten=True)

pair_meta = 'BrainPairCensus'
pair_annos = l1data.get_annotations_from_meta_annotations( pair_meta, flatten=True)
paired_ids = pair_nrns.ids()

ids_paired_left = list( set(all_left_ids).intersection(paired_ids) )
ids_paired_right = list( set(all_right_ids).intersection(paired_ids) )

reload(completeness)
match_report = completeness.match_report( ids_paired_left, ids_paired_right, match_via=pair_meta, CatmaidInterface=l1data, name1='Left', name2='Right', anno_reference = 'names', skip_annos=None, show_completeness=False )

From the adjacency matrix, get a list of paired homologous neuron->neuron edges only of completed neurons.

In [ ]:
A, skid_to_ind, ind_to_skid = pair_nrns.get_adjacency_matrix()

new_order = np.zeros((len(A)),dtype=np.int64)-1
curr_ind = 0
mirror_ind_start = len(ids_paired_left)
rel_skids = []

for ind, row in enumerate(match_report.iterrows()):
    if np.size(row[1]['Left']) > 0:
        new_order[curr_ind] = skid_to_ind[row[1]['Left']]
        new_order[mirror_ind_start+curr_ind] = skid_to_ind[row[1]['Right']]
        curr_ind += 1
new_order = new_order[np.where(new_order>=0)]

Anew = A[:,new_order][new_order]

norm_inds = np.arange(0,len(Anew))
mirror_inds = np.mod(np.arange(mirror_ind_start,mirror_ind_start+len(Anew)),len(Anew))

pair_mirror_dict = {'skid_to_ind':dict()}
pair_mirror_dict['skid_to_ind'] = {ind_to_skid[ind]:ii for ii,ind in enumerate(new_order)}
pair_mirror_dict['mirror_inds'] = mirror_inds

new_order = np.zeros((len(A)),dtype=np.int64)-1
curr_ind = 0
mirror_ind_start = len(ids_paired_left)
rel_skids = []

for aid in match_report:
    for ind, nid in enumerate(match_report[aid][0]):
        if ind < len(match_report[aid][1]):
            new_order[curr_ind] = skid_to_ind[nid]
            new_order[mirror_ind_start+curr_ind] = skid_to_ind[match_report[aid][1][ind]]
            curr_ind += 1
new_order = new_order[np.where(new_order>=0)]

Anew = A[:,new_order][new_order]

norm_inds = np.arange(0,len(Anew))
mirror_inds = np.mod(np.arange(mirror_ind_start,mirror_ind_start+len(Anew)),len(Anew))

pair_mirror_dict = {'skid_to_ind':dict()}
pair_mirror_dict['skid_to_ind'] = {ind_to_skid[ind]:ii for ii,ind in enumerate(new_order)}
pair_mirror_dict['mirror_inds'] = mirror_inds

pairs = []
for ind1 in np.arange(0,mirror_ind_start):
    for ind2 in norm_inds:
        w_norm = Anew[ind2,ind1]
        w_mirror = Anew[mirror_inds[ind2],mirror_inds[ind1]]
#         if w_norm > 0 and w_mirror > 0:
        pairs.append([ w_norm, w_mirror] )
pairs=np.array(pairs)

Fit the frequency of synapse counts both absolutely, and correlated between homologous edges. This lets us compute the likelihood ratio that a given pair of connections comes from matched versus independent draws, at least in a naive sense.

In [ ]:
Amatch_2d = sp.stats.gaussian_kde(
    np.concatenate( ( np.vstack((pairs[:,1],pairs[:,0])),np.vstack((pairs[:,0],pairs[:,1]))),axis=1),
    bw_method=4)
Afreq = sp.stats.gaussian_kde( pairs.reshape(1,pairs.size), bw_method=4)

max_syn = 100

norm_1d = 0
for n in np.arange(max_syn):
    norm_1d += Afreq(n)

Afreq_mat = np.zeros( (max_syn) )
for n in np.arange(max_syn):
    Afreq_mat[n] = Afreq(n)

Amatch_ratio_mat = np.zeros( (max_syn, max_syn) )
num_norm = 0
for n in np.arange(max_syn):
    for m in np.arange(max_syn):
        num_norm += Amatch_2d((n,m))

for n in np.arange(max_syn):
    for m in np.arange(max_syn):
        Amatch_ratio_mat[n,m] = np.log2( Amatch_2d((n,m))/num_norm + epsilon ) - np.log2( Afreq_mat[n]/norm_1d * Afreq_mat[m]/norm_1d + epsilon)


### Plot the Amatch_ratio_mat heatmap as a sanity check. You'd expect x~y to be the hottest areas, since that indicates similar connectivity.

In [ ]:
sns.heatmap(Amatch_ratio_mat[0:60,0:60], cmap="RdBu", center=0, vmin=-5, vmax=10)

#### Save the connectivity fits and pair map we generated for use in the homology-checking completeness workflow.

In [ ]:
import dill
with open('matched_synapse_prob','wb') as fid:
    dill.dump({'log_ratio': Amatch_ratio_mat, 'match':Amatch_2d, 'freq':Afreq},fid)

with open('pair_maps', 'wb') as fid:
    dill.dump({'pair_maps':pair_mirror_dict}, fid)